## 作業

實作本篇提到的三大概念

- 翻轉：實作上下左右的翻轉
- 縮放：比較鄰近差值與雙立方插值 (或雙線性插值) 的圖片品質
- 平移：建立 Translation Transformation Matrix 來做平移

In [50]:
import cv2
import numpy as np

# Using lena.png
img = 'img/lena.png'

# default read as cv2.IMREAD_COLOR(彩色圖)
img_bgr = cv2.imread(img, cv2.IMREAD_COLOR)

## 上下左右翻轉圖片

In [3]:
# vertical flip(垂直翻轉)
# img_bgr[h, w, channel], h reversed which means vertical flip
img_vflip = img_bgr[::-1, :, :]

# horizontal flip(水平翻轉)
# img_bgr[h, w, channel], w reversed which means horizontal flip
img_hflip = img_bgr[:, ::-1, :]

# vertical flip + horizontal flip(垂直+水平翻轉)
img_vhflip = img_bgr[::-1, ::-1, :]

# 縱向組合圖片
img_vflip_new = np.vstack((img_bgr, img_vflip))

# 橫向組合圖片
img_hflip_new = np.hstack((img_bgr, img_hflip))

# 橫向組合圖片(vhflip)
img_vhflip_new = np.hstack((img_bgr, img_vhflip))

while True :
    # display of img_vflip_new, named as 'vertical flip'
    cv2.imshow('vertical flip', img_vflip_new)
    # display of img_hflip_new, named as 'horizontal flip'
    cv2.imshow('horizontal flip', img_hflip_new)
    # display of img_vhflip_new, named as 'v+h flip'
    cv2.imshow('v+h flip', img_vhflip_new)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

## 縮放圖片

- 先透過縮小圖片去壓縮原有圖片保有的資訊，再放大比較不同方法之間的速度與圖片品質
- case1 : downscale and then upscale
- case2 : original image scale up directly 

In [71]:
# case1 : downscale and then upscale
from datetime import datetime

# (1)downscale to 20% of image
# second argument set to None, and fx depend on width, fy depend on height(or second argument set to (width, height))
img_down = cv2.resize(img_bgr, None, fx=0.2, fy=0.2)

# (2)upscale to 6 times from step(1), that means scale to 1.2 * original image
fx, fy = 6, 6

# 鄰近差值的插補法 + 計算花費時間
start_time = datetime.now()
img_up_NEAREST = cv2.resize(img_down, None, fx=fx, fy=fy, interpolation=cv2.INTER_NEAREST)
print('INTER_NEAREST zoom time consumed : {}'.format(datetime.now() - start_time))

# INTER_CUBIC的插補法 + 計算花費時間
start_time = datetime.now()
img_up_CUBIC = cv2.resize(img_down, None, fx=fx, fy=fy, interpolation=cv2.INTER_CUBIC)
print('INTER_CUBIC zoom time consumed : {}'.format(datetime.now() - start_time))

print(img_bgr.shape)        # original image
print(img_up_NEAREST.shape) # scaling up image

# 橫向組合圖片
img_up_new = np.hstack((img_up_NEAREST, img_up_CUBIC))

while True :
    # display of img_up_new, named as 'image zoom'
    cv2.imshow('image zoom', img_up_new)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

INTER_NEAREST zoom time consumed : 0:00:00.000998
INTER_CUBIC zoom time consumed : 0:00:00.002992
(512, 512, 3)
(612, 612, 3)


In [74]:
# case2 : original image scale up directly
from datetime import datetime

# (1)downscale to 20% of image
# second argument set to None, and fx depend on width, fy depend on height(or second argument set to (width, height))
img_down = cv2.resize(img_bgr, None, fx=0.2, fy=0.2)

# (2)upscale to 6 times from step(1), that means scale to 1.2 * original image
fx, fy = 6, 6

# 鄰近差值的插補法 + 計算花費時間
start_time = datetime.now()
img_up_NEAREST = cv2.resize(img_down, None, fx=fx, fy=fy, interpolation=cv2.INTER_NEAREST)
print('INTER_NEAREST zoom time consumed : {}'.format(datetime.now() - start_time))

# original image scale up directly (reaize to (width, height))
img_origin_up = cv2.resize(img_bgr, (img_up_NEAREST.shape[1], img_up_NEAREST.shape[0]))

# 橫向組合圖片
img_up_new = np.hstack((img_origin_up, img_up_NEAREST))

while True :
    # display of img_up_new, named as 'image zoom'
    cv2.imshow('image zoom', img_up_new)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

INTER_NEAREST zoom time consumed : 0:00:00.000996


## 平移幾何轉換

In [78]:
# 建立 Translation Transformation Matrix ([[1,0,tx], [0,1,ty]]), x move 100 pixels, y move 50 pixels
M = np.float32([[1,0,100], [0,1,50]])
# cv2.warpAffine(img, TT Matrix, dsize=(width, height))
img_shift = cv2.warpAffine(img_bgr, M, (img_bgr.shape[1], img_bgr.shape[0]))

# 橫向組合圖片
img_shift_new = np.hstack((img_bgr, img_shift))

while True :
    # display of img_shift_new, named as 'image shift'
    cv2.imshow('image shift', img_shift_new)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break